In [302]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import itertools
from collections import defaultdict
import io
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations

In [392]:
def gen_sols(n,m,quer_dict,noise):
    M = gp.Model()
    # M.Params.OutputFlag = 0
    
    y_var_dict = defaultdict()
    y_p_var_dict = defaultdict()
    y_n_var_dict = defaultdict()

    # Initialize the decision variables
    x = np.array([[M.addVar(vtype='B', name=f"x_{i}_{j}") 
                   for j in range(m)] for i in range(n)])
    # print(x)
    
    for quer in quer_dict.keys():
        y_var_dict[quer] = []
        y_p_var_dict[quer] = []
        y_n_var_dict[quer] = []
        
        positions = []
        for pos, att in enumerate(quer):
            if att == 1: 
                positions.append(pos)
        for i in range(n):
            y_var_dict[quer].append(M.addVar(vtype='B', name=f"y_{quer}_{i}"))
            y_p_var_dict[quer].append(M.addVar(vtype='B', name=f"y_p_{quer}_{i}"))
            y_n_var_dict[quer].append(M.addVar(vtype='B', name=f"y_n_{quer}_{i}"))
            
            M.addConstr( sum(quer)*y_var_dict[quer][i] + (sum(quer)+0.5)*y_p_var_dict[quer][i] <= sum([x[i][j] for j in positions]))
            M.addConstr( (sum(quer)-0.5)*y_n_var_dict[quer][i] + sum(quer)*y_var_dict[quer][i] + m*y_p_var_dict[quer][i] >= sum([x[i][j] for j in positions]))
            M.addConstr( y_var_dict[quer][i] + y_p_var_dict[quer][i] + y_n_var_dict[quer][i] == 1)
            
            # M.addConstr(sum([x[i][j] for j in positions]) <= sum(quer) + (m+1)*(1-y_var_dict[quer][i]))
            # M.addConstr(sum([x[i][j] for j in positions]) >= sum(quer) - (m+1)*(1-y_var_dict[quer][i]))
            
        if noise == 0:
            M.addConstr(sum(y_var_dict[quer]) == quer_dict[quer])
        else:
            M.addConstr(sum(y_var_dict[quer]) <= quer_dict[quer] + noise)
            M.addConstr(sum(y_var_dict[quer]) >= quer_dict[quer] - noise)
            
    # for i in range(n):
    #     M.addConstr()
       
    # obj_lst = []
#     for lst in y_var_dict.values():
#         obj_lst+=lst
        
#     M.setObjective(sum(obj_lst) , GRB.MINIMIZE)

#     # Add the constraints to the model
#     for i in range(i_max+1):
#         for j in range(j_max+1):
#             m.addConstr(x[i][j] <= out_table[i][j] +1)
#             m.addConstr(x[i][j] >= out_table[i][j] -1)

#     for i in range(i_max+1):
#         m.addConstr(sum(x[i,:][0:j_max]) == x[i][j_max])

#     for j in range(j_max+1):
#         m.addConstr(sum(x[:,j][0:i_max]) == x[i_max][j])

    # Parameters
    M.Params.PoolSearchMode = 2
    M.Params.PoolSolutions = 1
    # m.Params.PoolSolutions = 2
    M.Params.PoolGap = 0.0

    # Optimize
    M.optimize()
    
    out_lst = []


    for k in range(M.SolCount):
        M.Params.SolutionNumber = k
        out_x = np.zeros_like(x)
        for i in range(len(x)):
            for j in range(len(x[0])):
                out_x[i][j] = x[i][j].Xn
        # print([var.Xn for var in m.getVars()])
        out_lst.append(out_x)
        
    return out_x

In [289]:
def gen_bin_data_set(n,m):
    """n is number of people, m is number of attributes used, database is uniform random"""
    db = pd.DataFrame(np.random.randint(0,2,size=(n, m)), columns=[f'att_{x}' for x in range(m)])
    return db

In [420]:
def gen_queries_frac(m,frac,n_queries):
    """Generates random queries. Removes repeated queries"""
    
    queries = np.random.choice([0,1], size=[n_queries,m], p=[1-frac,frac])
    queries = list(set([tuple(x) for x in queries]))
    
    if tuple([0]*m) in queries:
        queries.remove(tuple([0]*m))
    return queries

In [291]:
def quer2string(query):
    string = ""
    for pos, att in enumerate(query):
        if att == 1:
            string += f'att_{pos} == 1 & '
    return string[0:-3]

In [411]:
def get_counts(db,quers,noise):
    """Returns noisy count of query, noise is uniform random integer over -noise to noise (inclusive). 
    Will perturb negative counts to 0."""
    out_dict = defaultdict(int)
    for query in quers:
        out_dict[tuple(query)] = max(len(db.query(quer2string(query))) + np.random.randint(-noise,noise+1),0)
    return out_dict

In [380]:
def binatodeci(binary):
    return sum(val*(2**idx) for idx, val in enumerate(reversed(binary)))

In [422]:
n = 10
m = 4
n_queries = 400
frac = 0.5
noise = 1
db = gen_bin_data_set(n,m)
display(db)
quers = gen_queries_frac(m,frac,n_queries)
# print(quers)
print(f"Number of Queries: {len(quers)}")
# print(quers)
quer_dict = get_counts(db,quers,noise)
# print(quer_dict)

#solving:
sol = np.array(gen_sols(n,m,quer_dict,noise), dtype=int) 

sol_tup = [tuple(x) for x in sol]
db_tup = [tuple(x) for x in list(db.to_numpy())]

print(sol_tup)
print(db_tup)

sol_tup, common = sol_tup[:], [ e for e in db_tup if e in sol_tup and (sol_tup.pop(sol_tup.index(e)) or True)]

print(f"{len(common)/len(sol_tup)*100:.2f}% Similarity")


sol_dict = {i:sol_tup.count(i) for i in set(sol_tup)}
db_dict = {i:db_tup.count(i) for i in set(db_tup)}
# print(db_dict)
print(sol_dict)
print(db_dict == sol_dict)

,att_0,att_1,att_2,att_3
0,1,1,0,1
1,1,0,1,0
2,0,0,1,0
3,1,1,1,1
4,1,0,0,1
5,0,1,1,0
6,0,0,1,0
7,1,0,1,1
8,0,1,0,0
9,1,0,0,0


Number of Queries: 15
Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 1
Set parameter PoolGap to value 0
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 480 rows, 490 columns and 2140 nonzeros
Model fingerprint: 0xc197177a
Variable types: 0 continuous, 490 integer (490 binary)
Coefficient statistics:
  Matrix range     [5e-01, 4e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 233 rows and 340 columns
Presolve time: 0.00s
Presolved: 247 rows, 150 columns, 1050 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.00000

In [305]:
print({'a': 3, 'b': 2} == {'b': 2, 'a': 3})

True


In [93]:
df = pd.DataFrame({'A': range(1, 6),
                   'B': range(10, 0, -2),
                   'C C': range(10, 5, -1)})
display(df)
display(df.query('10 == B and '))

,A,B,C C
0,1,10,10
1,2,8,9
2,3,6,8
3,4,4,7
4,5,2,6


,A,B,C C
0,1,10,10


In [112]:
lst = []
for pos, att in enumerate([0,1,0,1,0,0,1]):
    if att == 1: 
        lst.append(pos)
print(lst)

[1, 3, 6]


In [165]:
out = []
for lst in [[0,1,2],[2,3,4]]:
    out+=lst
out

[0, 1, 2, 2, 3, 4]

In [204]:
lst = [(1,0,0,0),(0,0,0,0),(1,0,0,0)]
lst = list(set([tuple(x) for x in lst]))
print(lst)
lst.remove((0,0,0,0))
print(lst)
print(tuple([1]*4))

[(1, 0, 0, 0), (0, 0, 0, 0)]
[(1, 0, 0, 0)]
[1, 1, 1, 1]


In [332]:
print(np.random.randint(-1,2))

1


In [405]:
l1 = [1,1,1,4,5]
l2 = [1,1,0,8,9]
 
l2, common = l2[:], [ e for e in l1 if e in l2 and (l2.pop(l2.index(e)) or True)]
common

[1, 1]